## Model Building and Evaluation

In this section, I performed the following:
* **Split the dataset into training and testing sets:** The dataset is divided into training (80%) and testing (20%) sets to ensure models are trained on one portion and tested on unseen data.
* **Train baseline models (Logistic Regression):** A simple Logistic Regression model is trained first as a benchmark to compare against more complex models.
* **Train advanced models (Random Forest, XGBoost):** More powerful models like Random Forest (ensemble-based) and XGBoost (gradient boosting) are trained to improve accuracy and performance.
* **Perform Cross-Validation:** Use cross-validation to assess model performance across different splits of the dataset.
* **Evaluate models and save the best-performing one:** Models are assessed using metrics such as classification report, confusion matrix, and AUC-ROC score. The best-performing model is saved for later use in deployment.
* **Plot Confusion Matrix for Best Models on Test Set:** Generate and visualize the confusion matrix for the best-performing model to analyze predictions.
* **Interactive Dashboard:** Interactive Dashboardthree tuned machine learning models (Logistic Regression, Random Forest, and XGBoost), computes classification metrics and AUC-ROC scores, and stores the results for visualization in an interactive dashboard

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import joblib

### Baseline Model

In [ ]:
# Logistic Regression
logistic_model = LogisticRegression(max_iter=500)
logistic_model.fit(X_train, y_train)

### Advanced Model

In [ ]:
# Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)


In [ ]:
# XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

### Hyperparameter Tuning

**Logistic Regression**

In [ ]:
# Logistic Regression Hyperparameter Tuning using GridSearchCV
logistic_params = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2']}
logistic_grid = GridSearchCV(LogisticRegression(max_iter=1000), logistic_params, cv=5, scoring='roc_auc', n_jobs=-1)
logistic_grid.fit(X_train, y_train)


In [ ]:
# Display the best parameters
print(f"Best Parameters: {logistic_grid.best_params_}")

# Display the best score
print(f"Best ROC-AUC Score: {logistic_grid.best_score_:.4f}")

**RandomForest Hyperparameter Tuning**

In [ ]:
# RandomForest Hyperparameter Tuning using RandomizedSearchCV

# Define hyperparameter search space
rf_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Use RandomizedSearchCV for faster tuning
rf_random = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_distributions=rf_params,
    n_iter=20,  # Number of random combinations to try
    cv=3,       # Reduce folds to speed up tuning
    scoring='roc_auc',
    random_state=42,
    n_jobs=-1
)

# Fit the model
rf_random.fit(X_train, y_train)

# Display best parameters and best score
print(f"Best Parameters: {rf_random.best_params_}")
print(f"Best ROC-AUC Score: {rf_random.best_score_:.4f}")

**xbg Hyperparameter Tuning**

In [ ]:
# xbg Hyperparameter Tuning using RandomizedSearchCV

# Define hyperparameter search space
xgb_params = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 1, 5],
    'min_child_weight': [1, 3, 5]
}

# Initialize the XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Use RandomizedSearchCV for faster hyperparameter tuning
xgb_random = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=xgb_params,
    n_iter=20,  # Number of random combinations to try
    scoring='roc_auc',  # Evaluation metric
    cv=5,  # Cross-validation folds
    n_jobs=-1,  # Use all available cores
    random_state=42,
    verbose=2  # Optional: Shows progress during tuning
)

# Fit the model to the training data
xgb_random.fit(X_train, y_train)

# Display the best parameters and best score
print(f"Best Parameters: {xgb_random.best_params_}")
print(f"Best ROC-AUC Score: {xgb_random.best_score_:.4f}")

### Cross-Validation

In [ ]:
# Evaluate with Cross-Validation
def evaluate_with_cross_validation(model, X_train, y_train):
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
    print(f"Cross-Validation AUC-ROC Scores: {scores}")
    print(f"Mean AUC-ROC Score: {scores.mean():.4f}\n")
    
# Cross-validate each tuned model
print("Logistic Regression Cross-Validation:")
evaluate_with_cross_validation(logistic_grid.best_estimator_, X_train, y_train)

print("Random Forest Cross-Validation:")
evaluate_with_cross_validation(rf_grid.best_estimator_, X_train, y_train)

print("XGBoost Cross-Validation:")
evaluate_with_cross_validation(xgb_grid.best_estimator_, X_train, y_train)


### Plot Confusion Matrix for Best Models on Test Set

In [ ]:
# Plot Confusion Matrix for Best Models on Test Set
for model, name in zip(
    [logistic_grid.best_estimator_, rf_grid.best_estimator_, xgb_grid.best_estimator_],
    ['Logistic Regression (Tuned)', 'Random Forest (Tuned)', 'XGBoost (Tuned)']
):
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    # Print classification metrics
    print(f"{name} Classification Report:")
    print(classification_report(y_test, y_pred))
    print(f"AUC-ROC Score: {roc_auc_score(y_test, y_proba):.4f}\n")

    # Generate confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    
    # Plot confusion matrix
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Not Fraud', 'Fraud'], yticklabels=['Not Fraud', 'Fraud'])
    plt.title(f"Confusion Matrix: {name}")
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.show()
    
    
# Save Confusion Matrices to CSV Files
for model, name in zip(
    [logistic_grid.best_estimator_, rf_grid.best_estimator_, xgb_grid.best_estimator_],
    ['logistic', 'rf', 'xgb']
):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    cm_df = pd.DataFrame(cm, index=["Not Fraud", "Fraud"], columns=["Not Fraud", "Fraud"])
    cm_df.to_csv(f"confusion_matrix_{name}.csv", index=False)

print("Confusion matrices saved as CSV files!")
    
import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score

# Initialize an empty list to store metrics
metrics = []

# Evaluate each model and collect metrics
for model, name in zip(
    [logistic_grid.best_estimator_, rf_grid.best_estimator_, xgb_grid.best_estimator_],
    ['Logistic Regression (Tuned)', 'Random Forest (Tuned)', 'XGBoost (Tuned)']
):
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    # Calculate metrics
    report = classification_report(y_test, y_pred, output_dict=True)
    auc_roc = roc_auc_score(y_test, y_proba)

    # Append metrics to the list
    metrics.append({
        "Model": name,
        "Accuracy": report['accuracy'],
        "Precision": report['1']['precision'],  # For the positive class (fraud)
        "Recall": report['1']['recall'],        # For the positive class (fraud)
        "F1-Score": report['1']['f1-score'],    # For the positive class (fraud)
        "ROC-AUC": auc_roc
    })

# Convert metrics to a DataFrame
metrics_df = pd.DataFrame(metrics)

# Save the metrics to a CSV file
metrics_df.to_csv("model_metrics.csv", index=False)

print("Model metrics saved as model_metrics.csv!")

## BUILDING INTERACTIVE DASHBOARD

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score

# Initialize an empty list to store metrics
metrics = []

# Evaluate each model and collect metrics
for model, name in zip(
    [logistic_grid.best_estimator_, rf_grid.best_estimator_, xgb_grid.best_estimator_],
    ['Logistic Regression (Tuned)', 'Random Forest (Tuned)', 'XGBoost (Tuned)']
):
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    # Calculate metrics
    report = classification_report(y_test, y_pred, output_dict=True)
    auc_roc = roc_auc_score(y_test, y_proba)

    # Append metrics to the list
    metrics.append({
        "Model": name,
        "Accuracy": report['accuracy'],
        "Precision": report['1']['precision'],  # For the positive class (fraud)
        "Recall": report['1']['recall'],        # For the positive class (fraud)
        "F1-Score": report['1']['f1-score'],    # For the positive class (fraud)
        "ROC-AUC": auc_roc
    })

# Convert metrics to a DataFrame
metrics_df = pd.DataFrame(metrics)

# Save the metrics to a CSV file
metrics_df.to_csv("model_metrics.csv", index=False)

print("Model metrics saved as model_metrics.csv!")

## Conclusion
The objective of predicting credit card payment defaults was analyzed using Logistic Regression, Random Forest, and XGBoost models. Below are the key takeaways:

### Logistic Regression:
- **Strengths:** High recall for non-defaults (0.97) and reasonable AUC-ROC score (0.7103).
- **Weaknesses:** Poor recall for defaults (0.24), missing many defaulting customers.

### Random Forest:
- **Strengths:** Improved recall for defaults (0.35) and a better AUC-ROC score (0.7750).
- **Weaknesses:** Still struggles with default recall and slightly more false positives.

### XGBoost:
- **Strengths:** Best AUC-ROC score (0.7795) and improved precision for defaults (0.61).
- **Weaknesses:** Recall for defaults remains low (0.37), slightly higher than Random Forest.

### Key Insight:
All models effectively identify non-defaulting customers but struggle with recall for defaults, which is critical for mitigating credit risk. XGBoost shows the best overall performance but requires further optimization to improve default classification.

### Recommendations:
- **Enhance feature engineering** (e.g., derive new features from payment history).
- **Balance the dataset** (e.g., using SMOTE or cost-sensitive learning).
- **Adjust classification thresholds** to improve recall for defaults while maintaining overall accuracy.